# The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
import glob

# The workers

In [2]:
ask_workers=8*28
memory='120GB'

In [3]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='pangeo',walltime='01:30:00',
                       job_extra=['--constraint=HSW24','--exclusive',
                                  '--nodes=1'],memory=memory,
                       interface='ib0') 
cluster.scale(ask_workers)
c= Client(cluster)
c



/scratch/cnt0024/hmg2840/albert7a/DEV/git/conda-pack/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34692 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://172.30.100.4:33743 Dashboard: http://172.30.100.4:34692/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 224, Cores=224, Memory=960.96 GB


# The data

In [6]:
files=sorted(glob.glob('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02*-S/*/eNATL60*gridT-2D*.nc'))
drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']
extra_coord_vars = []
chunks = dict(time_counter=240, y=240, x=480)
open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks,decode_cf=True,decode_times=False,
                   concat_dim="time_counter",combine='nested')


In [7]:
%time ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)

CPU times: user 3.41 s, sys: 309 ms, total: 3.71 s
Wall time: 6.24 s


In [8]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 1.02 s, sys: 25 ms, total: 1.04 s
Wall time: 1.03 s


In [ ]:
%time mean.load()

In [ ]:
cluster.close()